### This notebook is for debugging M2 faults

The user needs to provide a few timestamps (2nd cell), each corresponding to a point in time that we want to use in the comparison. Typically the first timestamp should be when the M2 was last known to be working properly.

The notebook goes through all the M2 telemetry, trying to imform the user whether certain info has changed before and after the system faulted.

In [ ]:
from openpyxl import load_workbook

from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import pandas as pd
import os

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
timeTitle = ['inclinometer', '85 deg', '75 deg', 'power cycle']
start = [Time('2021-03-25T15:00:00', scale = 'tai'), #8am
         Time('2021-03-25T16:10:00', scale = 'tai'), #9:10
         Time('2021-03-25T16:30:00', scale = 'tai'), #9:30
         Time('2021-03-25T18:04:00', scale = 'tai')] #11:04
end = []
nc = len(start) #number of comparison
for i in range(nc):
    end.append(start[i]+ timedelta(seconds=5)) #5s is the length of each data series.

In [ ]:
client = EfdClient('summit_efd')
csc_index = 1

aF = []
tF = []
temp = []
zA = []

aS = []
tS = []
aE = []
tE = []

p =[]
pI = []
disp = []

power = []
ilc = []

for i in range(nc):
    aF.append(await  client.select_time_series('lsst.sal.MTM2.axialForce', '*', start[i], end[i],  csc_index))
    tF.append(await  client.select_time_series('lsst.sal.MTM2.tangentForce', '*', start[i], end[i],  csc_index))
    temp.append(await  client.select_time_series('lsst.sal.MTM2.temperature', '*', start[i], end[i],  csc_index))
    zA.append(await  client.select_time_series('lsst.sal.MTM2.zenithAngle', '*', start[i], end[i],  csc_index))

    aS.append(await  client.select_time_series('lsst.sal.MTM2.axialActuatorSteps', '*', start[i], end[i],  csc_index))
    tS.append(await  client.select_time_series('lsst.sal.MTM2.tangentActuatorSteps', '*', start[i], end[i],  csc_index))
    aE.append(await  client.select_time_series('lsst.sal.MTM2.axialEncoderPositions', '*', start[i], end[i],  csc_index))
    tE.append(await  client.select_time_series('lsst.sal.MTM2.tangentEncoderPositions', '*', start[i], end[i],  csc_index))

    p.append(await  client.select_time_series('lsst.sal.MTM2.position', '*', start[i], end[i],  csc_index))
    pI.append(await  client.select_time_series('lsst.sal.MTM2.positionIMS', '*', start[i], end[i],  csc_index))
    disp.append(await  client.select_time_series('lsst.sal.MTM2.displacementSensors', '*', start[i], end[i],  csc_index))

    power.append(await  client.select_time_series('lsst.sal.MTM2.powerStatus', '*', start[i], end[i],  csc_index))
    ilc.append(await  client.select_time_series('lsst.sal.MTM2.ilcData', '*', start[i], end[i],  csc_index))

In [ ]:
#make sure we got data from each timestamp
for i in range(nc):
    print(len(tF[i]))

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(tF[a]['measured%d'%i])
    ax[a].set_title('tangent force measured, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(72):
        ax[a].plot(aF[a]['measured%d'%i])
    ax[a].set_title('axial force measured, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(tF[a]['lutGravity%d'%i])
    ax[a].set_title('tangent force lutG, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(72):
        ax[a].plot(aF[a]['lutGravity%d'%i])
    ax[a].set_title('axial force lutG, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(tF[a]['hardpointCorrection%d'%i])
    ax[a].set_title('tangent FB, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(72):
        ax[a].plot(aF[a]['hardpointCorrection%d'%i])
    ax[a].set_title('axial FB, %s'%timeTitle[a])

In [ ]:
a = 2
for i in range(72):
    if aF[a]['hardpointCorrection%d'%i][0]<20:
        print('%02d, %.2f'%(i+1,aF[a]['hardpointCorrection%d'%i][0]))
plt.plot([aF[a]['hardpointCorrection%d'%i][0] for i in range(72)],'.')
plt.grid()

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(12):
        ax[a].plot(temp[a]['ring%d'%i])
        ax[a].set_title('Ring temperature, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    ax[a].plot(zA[a]['measured']);
    ax[a].set_title('inclinometer reading, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(72):
        ax[a].plot(aS[a]['steps%d'%i])
        ax[a].set_title('axial actuator steps, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(72):
        ax[a].plot(aE[a]['position%d'%i])
        ax[a].set_title('axial encoder positions, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(tS[a]['steps%d'%i])
        ax[a].set_title('tangent actuator steps, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(tE[a]['position%d'%i])
        ax[a].set_title('tangent encoder positions, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for x in ['x','y','z', 'xRot', 'yRot', 'zRot']:
        ax[a].plot(p[a][x])
        ax[a].set_title('position, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for x in ['x','y','z', 'xRot', 'yRot', 'zRot']:
        ax[a].plot(pI[a][x])
        ax[a].set_title('positionIMS, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(6):
        ax[a].plot(disp[a]['deltaZ%d'%i])
        ax[a].plot(disp[a]['thetaZ%d'%i])
        ax[a].set_title('disp sensors, %s'%timeTitle[a])

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in range(78):
        j = np.mod(ilc[a]['status%d'%i],16)
        ax[a].plot(j)
        ax[a].plot( (ilc[a]['status%d'%i] - j)/16., '-.')
    ax[a].grid()
    ax[a].set_title('ILC status, %s'%timeTitle[a])

In [ ]:
a= 3 # No 1 out of nc
i = 0 #only look at first ILC, since all ILCs are the same in the above plot
j = np.mod(ilc[a]['status%d'%i],16)
print((ilc[a]['status%d'%i] - j)/16.) #this is communication counter

In [ ]:
fig, ax = plt.subplots(1,nc,figsize=(15,4))
for a in range(nc):
    for i in ['commVoltage', 'motorVoltage', 'commCurrent', 'motorCurrent']:
        ax[a].plot(power[a][i])
        ax[a].set_title('power status, %s'%timeTitle[a])

In [ ]:
for i in ['commVoltage', 'motorVoltage', 'commCurrent', 'motorCurrent']:
    for a in range(nc):
        print(i, a, power[a][i][0])